**Troubleshooting Scripts - Transaction Log**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Monitoring Availability Group Health and Performance**

Run on primary replica

In [ ]:
SELECT
    ar.replica_server_name as [Replica]
    ,DB_NAME(drs.database_id) AS DB
    ,drs.synchronization_state_desc as [Sync State]
    ,ars.synchronization_health_desc as [Health]
    ,ar.availability_mode as [Syncronous]
    ,drs.log_send_queue_size
    ,drs.redo_queue_size
    ,ISNULL(
        GhostReplicaState.max_low_water_mark_for_ghosts -
            drs.low_water_mark_for_ghosts,0
    ) AS [water_mark_diff]
    ,drs.log_send_rate
    ,drs.redo_rate
    ,pri.last_commit_time AS primary_last_commit_time
    ,IIF(drs.is_primary_replica = 1
        ,pri.last_commit_time
        ,drs.last_commit_time
    ) AS node_last_commit_time
    ,IIF(drs.is_primary_replica = 1
        ,0
        ,DATEDIFF(ms,drs.last_commit_time,pri.last_commit_time)
    ) AS commit_latency
FROM 
    sys.availability_groups ag WITH (NOLOCK) 
        JOIN sys.availability_replicas ar WITH (NOLOCK) ON 
            ag.group_id = ar.group_id
        JOIN sys.dm_hadr_availability_replica_states ars WITH (NOLOCK) ON 
            ar.replica_id = ars.replica_id
        JOIN sys.dm_hadr_database_replica_states drs WITH (NOLOCK) ON 
            ag.group_id = drs.group_id AND 
            drs.replica_id = ars.replica_id
        LEFT JOIN sys.dm_hadr_database_replica_states pri WITH (NOLOCK) ON 
            pri.is_primary_replica = 1 AND 
            drs.database_id = pri.database_id
        OUTER APPLY
         (
            SELECT MAX(drs2.low_water_mark_for_ghosts) AS 
                    max_low_water_mark_for_ghosts
            FROM sys.dm_hadr_database_replica_states drs2 WITH (NOLOCK)
            WHERE drs.database_id = drs2.database_id
        ) GhostReplicaState
WHERE	
    ars.is_local = 0
ORDER BY 
    replica_server_name, DB
OPTION (RECOMPILE, MAXDOP 1)

**Monitoring Availability Group Health and Performance** (SQL Server 2012)

Run on primary replica

In [ ]:
SELECT 
	ag.name AS [Availability Group]
	,ar.replica_server_name AS [Server]
	,DB_NAME(drs.database_id) AS [Database]
	,CASE WHEN ars.is_local = 1 THEN 'Local' ELSE 'Remote' END AS [DB Location]
	,ars.role_desc AS [Replica Role]
	,drs.synchronization_state_desc AS [Sync State]
	,ars.synchronization_health_desc AS [Health State]
	,drs.log_send_queue_size AS [Send Queue Size (KB)]
	,drs.log_send_rate AS [Send Rate KB/Sec]
	,drs.redo_queue_size AS [Redo Queue Size (KB)]
	,drs.redo_rate AS [Redo Rate KB/Sec]
	,drs.last_commit_time AS [Last Commit Time]
FROM 
	sys.availability_groups ag WITH (NOLOCK) 
		JOIN sys.availability_replicas ar WITH (NOLOCK) ON 
			ag.group_id = ar.group_id 
		JOIN sys.dm_hadr_availability_replica_states ars WITH (NOLOCK) ON 
			ar.replica_id = ars.replica_id
		JOIN sys.dm_hadr_database_replica_states drs WITH (NOLOCK) ON
			ag.group_id = drs.group_id and drs.replica_id = ars.replica_id
ORDER BY 
	ag.name, drs.database_id, ar.replica_server_name
OPTION (MAXDOP 1, RECOMPILE);

**Troubleshooting AG performance with xEvents**

Run during troublleshooting only. It is expensive trace. Start and stop manually watching and correlating live data

Session definition on primary node

In [ ]:
-- Primary node
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'AlwaysOn_Tracing_Primary')
	DROP EVENT SESSION AlwaysOn_Tracing_Primary ON SERVER;
GO

CREATE EVENT SESSION [AlwaysOn_Tracing_Primary] ON SERVER 
ADD EVENT sqlserver.hadr_capture_log_block,
ADD EVENT sqlserver.hadr_db_commit_mgr_harden,
ADD EVENT sqlserver.hadr_db_commit_mgr_harden_still_waiting,
ADD EVENT sqlserver.hadr_log_block_compression,
ADD EVENT sqlserver.hadr_log_block_send_complete,
ADD EVENT sqlserver.hadr_receive_harden_lsn_message,
ADD EVENT sqlserver.log_flush_complete,
ADD EVENT sqlserver.log_flush_start
ADD TARGET package0.ring_buffer(SET max_events_limit=(0),max_memory=(16384));

Session definition on secondary node

In [ ]:
-- Secondary node
IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'AlwaysOn_Tracing_Secondary')
	DROP EVENT SESSION AlwaysOn_Tracing_Secondary ON SERVER;
GO

CREATE EVENT SESSION [AlwaysOn_Tracing_Secondary] ON SERVER 
ADD EVENT sqlserver.hadr_apply_log_block,
ADD EVENT sqlserver.hadr_log_block_decompression,
ADD EVENT sqlserver.hadr_lsn_send_complete,
ADD EVENT sqlserver.hadr_send_harden_lsn_message,
ADD EVENT sqlserver.hadr_transport_receive_log_block_message,
ADD EVENT sqlserver.log_flush_complete,
ADD EVENT sqlserver.log_flush_start
ADD TARGET package0.ring_buffer(SET max_events_limit=(0),max_memory=(16384));